In [56]:
# pip install virtualenv
# pip install scrapy
# pip install crochet
# ! cd /2_LLMs
# ! virtualenv venv

In [ ]:
# ! scrapy
# !scrapy startproject bookscraper

Fuentes:
    
    
    https://www.mikulskibartosz.name/how-to-scrape-a-single-web-page-using-scrapy-in-jupyter-notebook/
    
Data:

    https://www.boletinoficial.gob.ar/detalleAviso/segunda/A0000001/20230503 (boletín digital)
    https://www.boletinoficial.gob.ar/detalleAviso/segunda/H681119/19830527 (boletín impreso/pdf)
    https://timeline.boletinoficial.gob.ar/ (búsqueda)
    https://www.boletinoficial.gob.ar/busquedaAvanzada/segunda
    
    scrapping boletin oficial

In [1]:
import os
import re
# scrape webpage
import scrapy
from scrapy.crawler import CrawlerRunner
# Reactor restart
from crochet import setup, wait_for


os.getcwd()
os.chdir('C:\\Users\\juani\\Documents\\3_My_Jupiter_Notebooks\\5_Galicia\\2_LLMs\\venv')

# CASO 1

In [2]:
setup()
class QuotesToCsv(scrapy.Spider):
    """scrape first line of  quotes from `wikiquote` by 
    Maynard James Keenan and save to json file"""
    name = "MJKQuotesToCsv"
    start_urls = [
        'https://en.wikiquote.org/wiki/Maynard_James_Keenan',
    ]
    custom_settings = {
        'ITEM_PIPELINES': {
            '__main__.ExtractFirstLine': 1
        },
        'FEEDS': {
            'quotes.csv': {
                'format': 'csv',
                'overwrite': True
            }
        }
    }

    def parse(self, response):
        """parse data from urls"""
        for quote in response.css('div.mw-parser-output > ul > li'):
            yield {'quote': quote.extract()}


class ExtractFirstLine(object):
    def process_item(self, item, spider):
        """text processing"""
        lines = dict(item)["quote"].splitlines()
        first_line = self.__remove_html_tags__(lines[0])

        return {'quote': first_line}

    def __remove_html_tags__(self, text):
        """remove html tags from string"""
        html_tags = re.compile('<.*?>')
        return re.sub(html_tags, '', text)

@wait_for(10)
def run_spider():
    """run spider with MJKQuotesToCsv"""
    crawler = CrawlerRunner()
    d = crawler.crawl(QuotesToCsv)
    return d

In [3]:
run_spider()

# CASO BOLETIN

In [5]:
class BoletinSpider(scrapy.Spider):
    name = "boletin"
    start_urls = [
        "https://www.boletinoficial.gob.ar/detalleAviso/segunda/A1194898/20230503"
    ]

    def parse(self, response):
        item = {}        
        item["title"] = response.xpath('//title/text()').get()
        item["css"] = response.css('div.avisoContenido article div#detalleAviso div#tituloDetalleAviso + div#cuerpoDetalleAviso p::text').get()
        
        print('title',item["title"])
        print('+css',item["css"])
        
        yield {"content": item}

def run_spider_BoletinSpider():
    """run spider with BoletinSpider"""
    crawler = CrawlerRunner()
    d = crawler.crawl(BoletinSpider)
    return d
run_spider_BoletinSpider()

In [22]:
import scrapy

class MySpider(scrapy.Spider):
    name = "my_spider"
    start_urls = []
    
    def start_requests(self):
        base_url = "https://www.boletinoficial.gob.ar/detalleAviso/segunda"
        a_values = ["A1194898", "A1234567", "A9876543"] # List of different values for A
        b_values = ["20230503", "20230430", "20230401"] # List of different values for B
        for a in a_values:
            for b in b_values:
                url = f"{base_url}/{a}/{b}"
                yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        # Extract data from the response object
        # For example:
        content = response.css('div.avisoContenido article div#cuerpoDetalleAviso p::text').get()
        yield {"content": content}
        
        
def run_spider_BoletinSpider():
    """run spider with BoletinSpider"""
    crawler = CrawlerRunner()
    d = crawler.crawl(MySpider)
    return d


run_spider_BoletinSpider()

In [2]:
# Please adapt the following code written in python using the libraries scrapy and crochet in order to save to a csv the scraped content of the website:

class BoletinSpider(scrapy.Spider):
    name = "boletin"
    start_urls = [
        "https://www.boletinoficial.gob.ar/detalleAviso/segunda/A1194898/20230503"
    ]

    def parse(self, response):
        item = {}        
        title = response.xpath('//title/text()').get()
        content = response.css('div.avisoContenido article div#detalleAviso div#tituloDetalleAviso + div#cuerpoDetalleAviso p::text').get()
        # content = response.css('div.avisoContenido article div#cuerpoDetalleAviso p::text').get()
        yield {"title": title, 'content':content}

def run_spider_BoletinSpider():
    """run spider with BoletinSpider"""
    crawler = CrawlerRunner()
    d = crawler.crawl(BoletinSpider)
    return d


test = run_spider_BoletinSpider()

C:\Users\juani\anaconda3\envs\llm\lib\site-packages\scrapy\utils\request.py:232: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation of the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting for information on how to handle this deprecation.
  return cls(crawler)


In [3]:
test

# CASO BOLETIN CON OUTPUT

In [ ]:
setup()
class BoletinToCsv(scrapy.Spider):
    """
    """
    name = "BoletinToCsv"
    start_urls = [
        'https://www.boletinoficial.gob.ar/detalleAviso/segunda/A1194898/20230503',
    ]
    custom_settings = {
        'ITEM_PIPELINES': {
            '__main__.ExtractFirstLine': 1
        },
        'FEEDS': {
            'quotes.csv': {
                'format': 'csv',
                'overwrite': True
            }
        }
    }

    def parse(self, response):
        """parse data from urls"""
        item = {}
        item["title"] = response.xpath('//title/text()').get()
        item["css"] = response.css('div.avisoContenido article div#detalleAviso div#tituloDetalleAviso + div#cuerpoDetalleAviso p::text').get()
        yield item


class ExtractFirstLine(object):
    def process_item(self, item, spider):
        """text processing"""
        lines = dict(item)["quote"].splitlines()
        first_line = self.__remove_html_tags__(lines[0])

        return {'quote': first_line}

    def __remove_html_tags__(self, text):
        """remove html tags from string"""
        html_tags = re.compile('<.*?>')
        return re.sub(html_tags, '', text)

@wait_for(10)
def run_spider():
    """run spider with MJKQuotesToCsv"""
    crawler = CrawlerRunner()
    d = crawler.crawl(QuotesToCsv)
    return d



class BoletinSpider(scrapy.Spider):
    name = "boletin"
    start_urls = [
        "https://www.boletinoficial.gob.ar/detalleAviso/segunda/A1194898/20230503"
    ]

    def parse(self, response):
        item = {}
        # title = response.xpath('//title/text()').get()
        item["title"] = response.xpath('//title/text()').get()
        # item["body"] = response.xpath('//div[@class="avisoContenido"]/article/div[@id="cuerpoDetalleAviso"]/p/text()').get()
        item["css"] = response.css('div.avisoContenido article div#detalleAviso div#tituloDetalleAviso + div#cuerpoDetalleAviso p::text').get()

        # item["body"] = response.xpath('//div[@id="tituloDetalleAviso"]').get()
        # print(item["title"])
        print('title',item["title"])
        print('css',item["css"])

        # for k,v in item.items():
        #     print(k,v)
        
        
        # product = response.css("div.product_main")
        # item["title"] = product.css("h1 ::text").extract_first()
        # item['category'] = response.xpath(
        #     "//ul[@class='breadcrumb']/li[@class='active']/preceding-sibling::li[1]/a/text()"
        # ).extract_first()
        # item['description'] = response.xpath(
        #     "//div[@id='product_description']/following-sibling::p/text()"
        # ).extract_first()
        
        
        

        
        return item
def run_spider_BoletinSpider():
    """run spider with MJKQuotesToCsv"""
    crawler = CrawlerRunner()
    d = crawler.crawl(BoletinSpider)
    return d
test = run_spider_BoletinSpider()

# CASO 2 tutorial

In [11]:
setup()

class BooksSpider(scrapy.Spider):
    name = 'books'

    def start_requests(self):
        url = 'https://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'
        yield scrapy.Request(url, callback=self.parse)

    def parse(self, response):
        item = {}
        product = response.css("div.product_main")
        item["title"] = product.css("h1 ::text").extract_first()
        item['category'] = response.xpath(
            "//ul[@class='breadcrumb']/li[@class='active']/preceding-sibling::li[1]/a/text()"
        ).extract_first()
        item['description'] = response.xpath(
            "//div[@id='product_description']/following-sibling::p/text()"
        ).extract_first()
        item['price'] = response.css('p.price_color ::text').extract_first()
        yield item
@wait_for(10)       
def run_spider2():
    """run spider with MJKQuotesToCsv"""
    crawler = CrawlerRunner()
    d = crawler.crawl(BooksSpider)
    return d

In [20]:
url = 'https://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'
product =  scrapy.Request(url)
product.css("div.product_main")

AttributeError: 'Request' object has no attribute 'css'